In [69]:
# a) Your Python code should create three predictors (Distance, Obstacle_Type, Angle_Approach) and one output variable (Time_Collision)

import os
import typing as t

import numpy as np
import pandas as pd

glass_df = pd.read_csv(os.path.join("data", "Group2_1.csv"))
concrete_df = pd.read_csv(os.path.join("data", "Group2_2.csv"))

# the only usecase for zip lol
for df, obs_type in zip([glass_df, concrete_df], ["Glass", "Concrete"]):
    # df.set_index("Time", drop=True, inplace=True)
    # df.reindex().sort_index(ascending=False, inplace=True)
    df["Obstacle_type"] = obs_type
    df["Angle_Approach"] = (10 if obs_type == "Concrete" else 0) + np.random.uniform(
        -5, 5, df.shape[0]
    )
    df["Time_Collision"] = df["Value"] / 24


concrete_df

,Time,Value,Obstacle_type,Angle_Approach,Time_Collision
0,16:49:49,214.17,Concrete,14.563217,8.923750
1,16:49:50,214.17,Concrete,13.970731,8.923750
2,16:49:50,214.17,Concrete,10.334131,8.923750
3,16:49:50,214.17,Concrete,13.332752,8.923750
4,16:49:50,470.47,Concrete,10.968287,19.602917
...,...,...,...,...,...
159,16:50:09,5.51,Concrete,11.212204,0.229583
160,16:50:09,5.91,Concrete,14.298141,0.246250
161,16:50:09,5.91,Concrete,9.863990,0.246250
162,16:50:09,5.91,Concrete,13.758312,0.246250


In [ ]:
# b) Your team can trim a few seconds of last rows of data where the same distance “Value” column in the datasets is exactly repeated.


def strip_similar_rows(
    df: pd.DataFrame,
    columns: str | list[str],
    side: t.Literal["start", "end", "both"] = "end",
    thresh=0.1,
) -> None:
    """
    THIS MUTATES THE DF

    strips the rows that are similar to the last row (acc to `tresh`),
    but it leaves the last row

    expects the dataframe to have an integer index


    args:
        - df (DataFrame): the dataframe to operate on
        - columns (str | list[str]): column(s) of the values to compare
        - side ("start", "end", "both"): where to strip rows, default "end"
        - tresh (float): += val to consider
    returns:
        - None

    """

    # WTF WAS THIS
    last = df["Value"].iloc[-1]
    rows_since_dupe = 0
    if side == "both":
        side = "end"
        strip_similar_rows(df, columns, "start", thresh)
    # iterrows() is a bad idea but whatever
    for i, val in (df[::-1] if side == "end" else df).iterrows():
        if not (last - thresh < val[columns] < last + thresh) or (
            i == df.index[-1] or i == df.index[0]
        ):
            rows_since_dupe += 1
            continue
        if rows_since_dupe == 2:
            break
        df.drop(index=i, inplace=True)  # dropping during iteration??? hehe


strip_similar_rows(glass_df, "Value", "end", 0.2)
strip_similar_rows(concrete_df, "Value", "end", 0.45)

,Time,Value,Obstacle_type,Angle_Approach,Time_Collision
0,16:40:57,189.37,Glass,3.215896,7.890417
1,16:40:58,186.22,Glass,-3.190447,7.759167
2,16:40:58,88.58,Glass,0.046723,3.690833
3,16:40:59,88.58,Glass,-4.271236,3.690833
4,16:40:59,88.58,Glass,-2.529870,3.690833
...,...,...,...,...,...
122,16:41:13,6.30,Glass,3.050773,0.262500
123,16:41:13,5.12,Glass,-1.272125,0.213333
124,16:41:13,4.33,Glass,0.562813,0.180417
125,16:41:13,3.94,Glass,0.791987,0.164167


''